In [1]:
from ngsolve import *

In [2]:
# from netgen.csg import Pnt,SplineCurve2d,CSGeometry,Revolution,Sphere
import numpy as np
import netgen.meshing as ngm
from netgen.csg import *
from netgen.meshing import MeshingStep
from ngsolve.comp import IntegrationRuleSpaceSurface

In [3]:
import sys
sys.path.append("./FEMGeoPackage")

In [ ]:
from geometry import *

In [5]:
from ngsolve.webgui import Draw

In [23]:
from netgen.occ import SplineApproximation, Pnt, Axis, Face, Wire, Segment, Revolve, OCCGeometry, Z, X, Y
#%% Construction of Initial Curved Mesh
dim = 3
order = 2
msize = 0.15
dt = Parameter(0)

## 2维曲面生成

For Red blood cell shape, we have the parametrization

$$
F(\theta, \varphi):=\left(\begin{array}{c}2 \cos \varphi \cos \theta \\ 2 \cos \varphi \sin \theta \\ \left(1-0.7\left(\cos ^2 \varphi-1\right)^2\right) \sin \varphi\end{array}\right), \quad \theta \in[0,2 \pi), \quad \varphi \in\left[-\frac{\pi}{2}, \frac{\pi}{2}\right]
$$


In [24]:
def Pos_Transformer(Pos_GF,dim=None):
    if dim is None:
        dim = Pos_GF.dim
    else:
        assert(Pos_GF.dim == dim)
    N = int(len(Pos_GF.vec)/dim)
    coords = Pos_GF.vec.Reshape(N).NumPy().copy()
    return coords.T

In [38]:
def CurveEllipsoid(phi): 
    z = 2.5*np.sin(phi)
    res = Pnt(2*np.cos(phi), 0, z)
    return res
def CurveRBC(phi): 
    z = (1-0.7*(np.cos(phi)**2-1)**2)*np.sin(phi)
    res = Pnt(2*np.cos(phi), 0, z)
    return res
def CliffordTorus(phi): 
    a = np.sqrt(1/2)
    z = a*np.sin(phi)
    res = Pnt(1-a*np.cos(phi), 0, z)
    return res

mesh = GetRotMesh(CurveRBC,msize)
# mesh = GetRotMesh(CliffordTorus,msize,T_min=0,T_max=2*np.pi,axis=Z,is_close=True)

In [39]:
mesh.Curve(2)

In [40]:
Draw(x,mesh,'x')

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

## 有限元空间

In [41]:
fes = H1(mesh,order=order)
fesV = VectorH1(mesh,order=order)

Disp = GridFunction(fesV)

In [42]:
vGf = GridFunction(fesV)

## Time stepping parameter

In [43]:
# Mixed space: (H,    V,    z,     nu)
fesMix =       fes * fes * fesV * fesV
# Grid functions
gfu     = GridFunction(fesMix)
gfuold  = GridFunction(fesMix)
# Previous time-step components
Hold, Vold, zold, nuold = gfuold.components

$$
Q = -\frac12 H^3 + |A|^2 H
$$

\begin{align*}
\int_{\Gamma[X]} \partial^{\bullet} H \varphi^H &-\int_{\Gamma[X]} \nabla_{\Gamma[X]} V \cdot \nabla_{\Gamma[X]} \varphi^H=-\int_{\Gamma[X]}|A|^2 V \varphi^H\\
\int_{\Gamma[X]} V \varphi^V & +\int_{\Gamma[X]} \nabla_{\Gamma[X]} H \cdot \nabla_{\Gamma[X]} \varphi^V=\int_{\Gamma[X]} Q \varphi^V \\
\int_{\Gamma[X]} \partial^{\bullet} \nu \cdot \varphi^\nu & -\int_{\Gamma[X]} \nabla_{\Gamma[X]} z \cdot \nabla_{\Gamma[X]} \varphi^\nu=\int_{\Gamma[X]}\left(H A-A^2\right) z \cdot \varphi^\nu \\
& +\int_{\Gamma[X]}\left(\left|\nabla_{\Gamma[X]} H\right|^2 \nu+A^2 \nabla_{\Gamma[X]} H\right) \cdot \varphi^\nu \\
& +2 \int_{\Gamma[X]}\left(A \nabla_{\Gamma[X]} H\right) \cdot\left(\nabla_{\Gamma[X]} \varphi^\nu \nu\right) \\
& +\int_{\Gamma[X]} Q \nabla_{\Gamma[X]} \cdot \varphi^\nu-\int_{\Gamma[X]} Q H \nu \cdot \varphi^\nu \\
\int_{\Gamma[X]} z \cdot \varphi^z & +\int_{\Gamma[X]} \nabla_{\Gamma[X]}\nu \cdot \nabla_{\Gamma[X]} \varphi^z=\int_{\Gamma[X]}|A|^2 \nu \cdot \varphi^z
\end{align*}

In [44]:
SD_opt = False
H , V , z, nu = fesMix.TrialFunction()
Ht, Vt, zt, nut = fesMix.TestFunction()

## Weingarten Map A 通过上一层的法向量（有限元函数）来计算
A = grad(nuold).Trace()
A = 1/2*(A.trans+A)
if SD_opt:
    Q = 0
else:
    Q = -1/2*Hold**3+InnerProduct(A,A)*Hold

# 归一化对于长时间演化会有帮助。如果没有归一化，可能会出现持续膨胀的情况
nuold_uni = nuold/Norm(nuold)
Lhs = BilinearForm(fesMix,symmetric=False)
Rhs = LinearForm(fesMix)
## ds may blow up
# H的演化方程离散
Lhs += (1/dt*H*Ht - InnerProduct(grad(V).Trace(),grad(Ht).Trace()))*ds 
Lhs += InnerProduct(A,A)*V*Ht*ds
Rhs += 1/dt*Hold*Ht*ds 
# V的弱形式
Lhs += (V*Vt + InnerProduct(grad(H).Trace(),grad(Vt).Trace()))*ds
Rhs += Q*Vt*ds
# n的演化方程离散
Lhs += 1/dt*InnerProduct(nu,nut)*ds - InnerProduct(grad(z).Trace(),grad(nut).Trace())*ds
Rhs += 1/dt*InnerProduct(nuold_uni,nut)*ds
Lhs += (- Hold*InnerProduct(A*z,nut) + InnerProduct(A*(A*z),nut))*ds
Rhs += (InnerProduct(grad(Hold).Trace(),grad(Hold).Trace())*InnerProduct(nuold_uni,nut)\
    + InnerProduct(A*(A*grad(Hold).Trace()),nut)\
    + 2*InnerProduct(grad(nut).Trace()*(A*grad(Hold).Trace()), nuold_uni))*ds
Rhs += Q*Trace(grad(nut).Trace())*ds - Q*Hold*InnerProduct(nuold_uni,nut)*ds
# z的弱形式
Lhs += (InnerProduct(z,zt) + InnerProduct(grad(nu).Trace(),grad(zt).Trace()))*ds
Rhs += (InnerProduct(A,A)*InnerProduct(nuold_uni,zt))*ds
lhs = Lhs
rhs = Rhs

## 显示网格移动的设定

In [45]:
SetVisualization(deformation=True)

## 时间演化

In [46]:
mesh.UnsetDeformation()
Disp.Set(CF((0,0,0)),definedon=mesh.Boundaries(".*"))
nuold.Set(-specialcf.normal(3),definedon=mesh.Boundaries(".*"))
Hold.Set(Trace(-specialcf.Weingarten(3)),definedon=mesh.Boundaries(".*"))

In [47]:
mesh.UnsetDeformation()
Draw(Norm(Hold), mesh, 'Curvature')

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [48]:
t_old = 0

In [49]:
tau0 = 0.025
dt.Set(tau0)

In [50]:
sceneu = Draw(Norm(Hold), mesh, 'disp')
Reset = True
while t_old<5:
    mesh.SetDeformation(Disp)
    lhs.Assemble()
    rhs.Assemble()
    gfu.vec.data = lhs.mat.Inverse(inverse="umfpack")*rhs.vec
    vGf.Interpolate(gfu.components[1]*gfu.components[-1],definedon=mesh.Boundaries(".*"))
        
    gfuold.vec.data = BaseVector(gfu.vec.FV().NumPy())
    Disp.vec.data = BaseVector(Disp.vec.FV().NumPy() + tau0*vGf.vec.FV().NumPy())
    sceneu.Redraw()
    t_old += tau0

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

KeyboardInterrupt: 